In [15]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from speech_modeling import h_model, h_input
raw_df = pd.read_csv("./formants.csv")

input_data = h_input(raw_df)
input_data.process('region')
input_data.select_features(selected_features=['F1mF2', 'duration'])
input_data.normalize(method='minmax')
input_data.select_places(places=['uk', 'texas', 'california', 'newyork'])

model_data = input_data.input_df
model_features = ['age', 'gender'] + list(input_data.input_df.columns[4:])

m = h_model(data=model_data, features=model_features, y_feature="location", y_main="uk")
m.fit(NUM_SAMPLES=5)

Fitting Model of type rforest
Modeling Sample 1 of 5
Modeling Sample 2 of 5
Modeling Sample 3 of 5
Modeling Sample 4 of 5
Modeling Sample 5 of 5

Accuracy: 0.912

Variable Importance Measurements:
          Feature  Importance
5       F1mF2-ask    0.000251
74        du-frog    0.000207
65         du-bob    0.000178
99       du-store    0.000133
31   F1mF2-please    0.000133
68        du-call    0.000104
34      F1mF2-she    0.000089
4       F1mF2-and    0.000089
58        du-also    0.000074
104      du-three    0.000059
